In [1]:
import numpy as np
import pandas as pd

In [2]:
df_faf2012=pd.read_csv("faf35_provisional_2012_state.csv")

In [3]:
df_faf2012=df_faf2012.drop(["curval_2012"],axis=1)


In [4]:
df_faf2012.shape

(549132, 12)

In [5]:
df_faf2012=df_faf2012[df_faf2012['value_2012']!=0]

In [6]:
df_faf2012.shape

(501231, 12)

In [7]:
df_faf2012.mode()

,fr_orig,dms_origst,dms_destst,fr_dest,fr_inmode,dms_mode,fr_outmode,sctg2,trade_type,value_2012,tons_2012,tmiles_2012
0,801.0,6,6,801.0,3.0,1,3.0,34,2,0.0001,0.0,0.0


Most DOmestically produced goods-California
Domestic DOnation Goods - California
MOst Foreign Imports COuntry-  CANADA
Most Foreign EXports Country - CANADA
Domestic MODE Preffered- Truck
fr_inmode,fr_outmode-Water
Commodity which gets exported/imported most-Machinery


In [8]:
df_faf2012=df_faf2012.fillna(0)

In [9]:
df_faf2012_F=df_faf2012.iloc[:,0:9]

In [10]:
df_faf2012_Transform=df_faf2012[["value_2012","tons_2012","tmiles_2012"]]

In [11]:
df_faf2012_Transform=df_faf2012_Transform.transform(func = ['sqrt'])

In [12]:
df_faf2012_Transform=pd.concat([df_faf2012_F,df_faf2012_Transform],axis=1)

In [13]:
df_faf2012_Transform

,fr_orig,dms_origst,dms_destst,fr_dest,fr_inmode,dms_mode,fr_outmode,sctg2,trade_type,"(value_2012, sqrt)","(tons_2012, sqrt)","(tmiles_2012, sqrt)"
0,0.0,1,1,0.0,0.0,1,0.0,1,1,36.033584,34.729116,10.964678
1,0.0,1,1,0.0,0.0,1,0.0,2,1,22.043666,50.602931,15.485686
2,0.0,1,1,0.0,0.0,1,0.0,3,1,32.115590,52.907020,15.447254
3,0.0,1,1,0.0,0.0,1,0.0,4,1,27.444313,55.639712,16.641353
4,0.0,1,1,0.0,0.0,1,0.0,5,1,46.159158,30.195124,8.189547
...,...,...,...,...,...,...,...,...,...,...,...,...
549127,808.0,53,56,0.0,5.0,1,0.0,26,2,0.044721,0.020000,0.019901
549128,808.0,53,56,0.0,5.0,1,0.0,30,2,0.050000,0.010000,0.009940
549129,808.0,53,56,0.0,5.0,1,0.0,33,2,0.063246,0.043589,0.043388
549130,808.0,53,56,0.0,5.0,1,0.0,34,2,0.432319,0.186279,0.183264


In [14]:
df_faf2012_Transform=df_faf2012_Transform.rename(columns={('value_2012', 'sqrt'): "value_2012_sqrt",('tons_2012', 'sqrt'):"Tons_2012_sqrt",('tmiles_2012', 'sqrt'):"tmiles_2012_sqrt"})
import seaborn as sns

In [15]:
#sns.pairplot(df_faf2012_Transform)

In [16]:
import random
df_faf2012_Trans_sample=df_faf2012_Transform.sample(n=50000)

In [17]:
X=df_faf2012_Trans_sample.drop(["value_2012_sqrt"],axis=1)
y=df_faf2012_Trans_sample["value_2012_sqrt"]

In [18]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

In [19]:
models = []
models.append(('KNN', KNeighborsRegressor()))
models.append(('RF', RandomForestRegressor()))
models.append(('SVR', SVR()))
from sklearn import model_selection

In [ ]:
names = []
scores = []
Bias_ERROR=[]
VAR_ERROR=[]

for name, model in models:
    
    kfold=model_selection.KFold(shuffle=True,n_splits=5,random_state=10)
    CV_results=model_selection.cross_val_score(model,X,y,cv=kfold,scoring="r2")
    
    names.append(name)
    scores.append(CV_results.mean())
    Bias_ERROR.append(1-CV_results.mean())
    VAR_ERROR.append(np.std(CV_results,ddof=1))
kf_cross_val = pd.DataFrame({'Name': names, 'Score': scores,"AVG_BIAS ERROR":Bias_ERROR,"VAR_ERROR":VAR_ERROR})


In [ ]:
kf_cross_val

In [ ]:
from sklearn.neighbors import KNeighborsRegressor


In [ ]:
values=[]
reg_bias=[]
reg_var=[]
for val in np.arange(1,101):
    neigh = KNeighborsRegressor(n_neighbors=val)
    kfold=model_selection.KFold(n_splits=15,shuffle=True,random_state=5)
    cv_results=model_selection.cross_val_score(neigh,X,y,scoring="r2",cv=kfold)
    reg_bias.append(1-np.mean(cv_results))
    reg_var.append(np.std(CV_results,ddof=1))
    values.append(val)
    
GBC_bias=pd.DataFrame({"Estimators":values,"B.E":reg_bias})
GBC_reg=pd.DataFrame({"Estimators":values,"B.E":reg_var})

In [ ]:
GBC_bias[GBC_bias["B.E"]==GBC_bias["B.E"].min()]

In [ ]:
print("r2 score ",GBC_bias["B.E"].min())

In [ ]:
GBC_reg[GBC_reg["B.E"]==GBC_reg["B.E"].min()]

In [20]:
  from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(df_faf2012_Transform.values, i) for i in range(df_faf2012_Transform.shape[1])]
VIF_Housing=pd.DataFrame({'vif': vif}, index=df_faf2012_Transform.columns)
VIF_Housing.sort_values(by='vif')

,vif
value_2012_sqrt,2.020855
dms_mode,2.986112
tmiles_2012_sqrt,3.135927
Tons_2012_sqrt,3.516882
sctg2,4.495563
dms_destst,4.567696
dms_origst,4.721017
fr_inmode,7.605800
fr_outmode,8.993510
fr_orig,19.133915


In [21]:
df_faf2012_Transform=df_faf2012_Transform.drop(["trade_type"],axis=1)

In [22]:
vif = [variance_inflation_factor(df_faf2012_Transform.values, i) for i in range(df_faf2012_Transform.shape[1])]
VIF_Housing=pd.DataFrame({'vif': vif}, index=df_faf2012_Transform.columns)
VIF_Housing.sort_values(by='vif')

,vif
value_2012_sqrt,1.987771
dms_mode,2.548141
tmiles_2012_sqrt,3.074622
Tons_2012_sqrt,3.505069
sctg2,3.622993
dms_destst,3.706070
dms_origst,3.761348
fr_inmode,7.588966
fr_orig,8.433890
fr_outmode,8.880951


In [23]:
df_faf2012_Transform=df_faf2012_Transform.drop(["fr_dest"],axis=1)
vif = [variance_inflation_factor(df_faf2012_Transform.values, i) for i in range(df_faf2012_Transform.shape[1])]
VIF_Housing=pd.DataFrame({'vif': vif}, index=df_faf2012_Transform.columns)
VIF_Housing.sort_values(by='vif')

,vif
fr_outmode,1.856300
value_2012_sqrt,1.980897
dms_mode,2.503938
tmiles_2012_sqrt,3.074101
Tons_2012_sqrt,3.505057
sctg2,3.581581
dms_destst,3.635536
dms_origst,3.701084
fr_inmode,7.588951
fr_orig,8.356319


In [24]:
X=df_faf2012_Transform.drop(["value_2012_sqrt"],axis=1)
y=df_faf2012_Transform["value_2012_sqrt"]

In [25]:
from sklearn.ensemble import AdaBoostRegressor
regressor=RandomForestRegressor(n_estimators=12,random_state=0)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
values=[]
ADA_BIAS=[]
ADA_VAR=[]
ADA=AdaBoostRegressor(base_estimator=regressor,n_estimators=20,random_state=0)
kfold=model_selection.KFold(n_splits=5,shuffle=True,random_state=5)
cv_results=model_selection.cross_val_score(ADA,X,y,scoring="r2",cv=kfold)
ADA_BIAS.append(1-np.mean(cv_results))
ADA_VAR.append(np.std(cv_results,ddof=1))

    
ADA_BOOST=pd.DataFrame({"R_2 score":np.mean(cv_results),"B.E":ADA_BIAS,"V.E":ADA_VAR})

NameError: name 'ADA_BOOST' is not defined

In [2]:
df_faf2012

NameError: name 'df_faf2012' is not defined